In [ ]:
import csv
import pandas as pd
import itertools
import math
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
import string
import re
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import classification_report, plot_confusion_matrix
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\David_Cruz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
news = pd.read_csv('news.csv')

In [ ]:
news.head()

,id,title,score,subreddit,url,num_comments,text,created,police_brutality
0,dfn3yi,Blizzard Employees Staged a Walkout After the ...,226324,news,https://www.thedailybeast.com/blizzard-employe...,9623,NaN,1.570683e+09,0
1,eubjfc,Kobe Bryant killed in helicopter crash in Cali...,213684,news,https://www.fox5dc.com/news/kobe-bryant-killed...,20705,NaN,1.580096e+09,0
2,84aebi,Scientist Stephen Hawking has died aged 76,188175,news,http://news.sky.com/story/scientist-stephen-ha...,6916,NaN,1.521028e+09,0
3,dp5lr1,Jeffrey Epstein's autopsy more consistent with...,186243,news,https://www.foxnews.com/us/forensic-pathologis...,10057,NaN,1.572465e+09,0
4,7ej943,F.C.C. Announces Plan to Repeal Net Neutrality,178001,news,https://www.nytimes.com/2017/11/21/technology/...,10857,NaN,1.511312e+09,0


In [ ]:
news=news.drop(['id', 'score','subreddit', 'url','num_comments','text','created'], axis=1)


In [ ]:
news.head()
print(news.shape)

(993, 2)


In [ ]:
pol_uof = pd.read_csv('police_use_force.csv')

In [ ]:
pol_uof['title'] = pol_uof['name']

In [ ]:
pol_uof = pol_uof.drop(['name'], axis = 1)

In [ ]:
pol_uof

,police_brutality,title
0,1,Compilation of incidents
1,1,Police shove a protesting woman down a set of ...
2,1,Police throw tear-gas at protesters on a bridge.
3,1,Police assault protesters
4,1,Police shoot non-violent protester in the head
...,...,...
990,1,Reporter shows tear gas canister fired at him ...
991,1,Woman bleeding from face after being shot by p...
992,1,"Police Mace, shoot pepper bullets at protester..."
993,1,Peaceful protesters arrested for breaking curfew


In [ ]:
frames = [news, pol_uof]

In [ ]:
df = pd.concat(frames)

In [ ]:
df.shape

(1988, 2)

In [ ]:
df.head()

,title,police_brutality
0,Blizzard Employees Staged a Walkout After the ...,0
1,Kobe Bryant killed in helicopter crash in Cali...,0
2,Scientist Stephen Hawking has died aged 76,0
3,Jeffrey Epstein's autopsy more consistent with...,0
4,F.C.C. Announces Plan to Repeal Net Neutrality,0


In [ ]:
df['police_brutality'].value_counts()

1    995
0    993
Name: police_brutality, dtype: int64

In [ ]:
995+993

1988

In [ ]:
# This is baseline accuracy of 50%
995/1988

0.5005030181086519

In [ ]:
# create a column kfold so we know which fold . set to -1
df['kfold'] = -1

In [ ]:
#randomize the rows
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# fetch labels
y = df.police_brutality.values

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=5)

In [ ]:
# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

In [ ]:
# go over the folds created
for fold_ in range(5):
    # temp dataframes for train and test
    train_df= df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)
    
    #initialize countvectorizer with NLTK's work_tokenize
    #function as tokenizer
    count_vec = CountVectorizer(tokenizer=word_tokenize,
                               token_pattern=None)

    #fit the count_vectorizor on training data titles
    count_vec.fit(train_df.title)

    # transform training and validation data
    xtrain = count_vec.transform(train_df.title)
    xtest = count_vec.transform(test_df.title)

    # initialize logistic regression model
    model = linear_model.LogisticRegression()

    # fit the model on training titles and police brutality rating
    model.fit(xtrain, train_df.police_brutality)

    #make predictions on test data
    #baseline was ~50%
    pred = model.predict(xtest)

    #accuracy
    accuracy = metrics.accuracy_score(test_df.police_brutality, pred)

    #print results
    print(f"Fold: {fold_}")
    print(f"Accuracy = {accuracy}")
    print("")

Fold: 0
Accuracy = 0.9723618090452262

Fold: 1
Accuracy = 0.957286432160804

Fold: 2
Accuracy = 0.964824120603015

Fold: 3
Accuracy = 0.9622166246851386

Fold: 4
Accuracy = 0.9697732997481109



In [ ]:
test_title = ["""Aliens bake humungous souffle"""]

x = count_vec.transform(test_title)

In [ ]:
model.predict(x)

array([0], dtype=int64)